In [4]:
import numpy as np
import pandas as pd
import cv2
from random import randint
import os
from keras import backend as K
from metrics import dice_coef
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")
from helpers import create_data_set_from_generator, get_train_generator
from losers import dice_loss
from sklearn.model_selection import train_test_split
from util import join_paths
from skimage.transform import resize
import tensorflow as tf
# from keras.preprocessing.image import load_img
# from keras import Model
# #from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
# from keras.models import load_model
# from tensorflow.keras.callbacks import EarlyStopping
# from keras.optimizers import Adam
# from keras.utils.vis_utils import plot_model
# from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout

from tqdm import tqdm_notebook

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import cv2
from util import join_paths

def get_train_generator(directory, mask_dir, img_dir, read_type=cv2.IMREAD_GRAYSCALE):
    def train_generator():
        _img_dir = join_paths(directory, img_dir)
        _mask_dir = join_paths(directory, mask_dir)

        for imag_file in os.listdir(_img_dir):
            x_img = cv2.imread(join_paths(_img_dir, imag_file), read_type)
            y_img = cv2.imread(join_paths(_mask_dir, imag_file), read_type)
            x_img = np.pad(x_img, ((13, 14), (13, 14)), mode='symmetric').astype(np.float64) / 255.0
            y_img = np.pad(y_img, ((13, 14), (13, 14)), mode='symmetric').astype(np.float64) / 255.0
            img_shape = (x_img.shape[0], x_img.shape[1], 1)
            yield (x_img.reshape(img_shape), y_img.reshape(img_shape))

    return train_generator


def get_test_generator(directory, img_dir, read_type=cv2.IMREAD_GRAYSCALE):
    def test_generator():
        _img_dir = join_paths(directory, img_dir)
        for imag_file in os.listdir(_img_dir):
            x_img = cv2.imread(join_paths(_img_dir, imag_file), read_type)
            yield x_img

    return test_generator


def create_data_set_from_generator(generator, _types, _shapes, buffer_size=100):
    ds = tf.data.Dataset.from_generator(generator, _types, _shapes)
    ds = ds.prefetch(buffer_size)
    return ds


def get_channels(_type):
    if _type == cv2.IMREAD_GRAYSCALE:
        return 1
    elif _type == cv2.IMREAD_COLOR:
        return 3
    return 1


def create_tfrecord(image_mask_list: [()], tfrecord_file, preprocess_callbacks: [callable] = None,
                    _type=cv2.IMREAD_GRAYSCALE):
    with tf.io.TFRecordWriter(tfrecord_file) as writer:
        for img_file, mask_file in image_mask_list:
            ser_img = serialize_tgs_image(img_file, mask_file, preprocess_callbacks, _type)
            writer.write(ser_img.SerializeToString())


def pad_images(img, mask, pad_shapes=((13, 14), (13, 14))):
    img = np.pad(img, pad_shapes, mode='symmetric').astype(np.float32) / 255.0
    mask = np.pad(mask, pad_shapes, mode='symmetric').astype(np.float32) / 255.0
    return img, mask


def reshape_img(img, mask, shape=(128, 128, 1)):
    img = np.resize(img, shape)
    mask = np.resize(mask, shape)
    return img, mask


def serialize_tgs_image(img_file, mask_file, preprocess_callbacks: [callable], _type):
    img = cv2.imread(img_file, _type)
    mask = cv2.imread(mask_file, _type)
    if preprocess_callbacks:
        for preprocess_callback in preprocess_callbacks:
            img, mask = preprocess_callback(img, mask)

    img = img.astype(np.float32)
    mask = mask.astype(np.float32)

    ser_image = tf.train.Example(features=tf.train.Features(feature={
        'img': tf.train.Feature(float_list=tf.train.FloatList(value=img.reshape(-1).tolist())),
        'mask': tf.train.Feature(float_list=tf.train.FloatList(value=mask.reshape(-1).tolist()))
    }))
    return ser_image


def get_train_val_paths(directory, imgs_dir, masks_dir, percentage=0.8):
    _img_dir = join_paths(directory, imgs_dir)
    _mask_dir = join_paths(directory, masks_dir)
    images = [(join_paths(_img_dir, img_file), join_paths(_mask_dir, img_file)) for img_file in os.listdir(_img_dir)]
    train_size = int(percentage * len(images))
    return images[:train_size], images[train_size:]


def create_deserializer(shape=(128, 128, 1)):
    def deserialize_tgs_image(tfrecord):
        features = {
            'img': tf.FixedLenFeature(shape, tf.float32),
            'mask': tf.FixedLenFeature(shape, tf.float32)
        }
        sample = tf.parse_single_example(tfrecord, features)
        img = sample['img']
        mask = sample['mask']
        return tf.cast(img, tf.float64), tf.cast(mask, tf.float64)
    return deserialize_tgs_image


def create_dataset_from_tfrecord(tf_records, decode_func):
    dataset = tf.data.TFRecordDataset(tf_records)
    dataset = dataset.map(decode_func)
    dataset = dataset.prefetch(64)
    return dataset


    # train_ds = create_dataset_from_tfrecord(['train_images.tfrecord'], create_deserializer())
    # start_time = time.time()
    # for batch in train_ds.take(3200):
    #     cv2.imshow('img', (batch[0].numpy() * 255.0).astype(np.uint8))
    #     cv2.imshow('mask', (batch[1].numpy() * 255.0).astype(np.uint8))
    #     cv2.waitKey()
    # print('time elapsed {}'.format(time.time() - start_time))
    #
    # ds_train = create_data_set_from_generator(get_train_generator('./tgs/train', mask_dir='masks', img_dir='images'),
    #                                           _types=(tf.float64, tf.float64),
    #                                           _shapes=(tf.TensorShape([128, 128, 1]), tf.TensorShape([128, 128, 1])))
    # start_time = time.time()
    # for batch in ds_train.take(3200):
    #     print('.', end='')
    # print('')
    # print('time elapsed {}', format(time.time()-start_time))
# train_files, validation_files = get_train_val_paths('drive/My Drive/Colab Notebooks/train', masks_dir='masks', imgs_dir='images')
# print(len(train_files))
# print(len(validation_files))
# create_tfrecord(train_files, 'train_images.tfrecord', [reshape_img])
# create_tfrecord(validation_files, 'validation_images.tfrecord', [reshape_img])


In [0]:
train_ds = create_dataset_from_tfrecord(['train_images.tfrecord'], create_deserializer())
valid_ds = create_dataset_from_tfrecord(['validation_images.tfrecord'], create_deserializer())

# Layers#

In [0]:
class UnetEncodeLayer(tf.keras.layers.Layer):

    def __init__(self, kernel_size, filters, pool_size=(2, 2), dropout_value=0.25):
        super(UnetEncodeLayer, self).__init__()

        self.dropout_value = dropout_value
        self.layer1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', activation='relu')
        self.layer2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', activation='relu')
        self.pool = tf.keras.layers.MaxPool2D(pool_size)
        self.dropout = tf.keras.layers.Dropout(self.dropout_value)

    def call(self, inputs):
        _out = self.layer1(inputs)
        _out = self.layer2(_out)
        _out_pool = self.pool(_out)

        return _out, self.dropout(_out_pool)


class UnetDecodeLayer(tf.keras.layers.Layer):

    def __init__(self, kernel_size, filters, dropout_value=0.25, strides=(2, 2)):
        super(UnetDecodeLayer, self).__init__()

        self.deconv_layer = tf.keras.layers.Conv2DTranspose(filters=filters, kernel_size=kernel_size, padding='same', strides=strides)
        self.dropout = tf.keras.layers.Dropout(dropout_value) if dropout_value is not None else None
        self.layer1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu')
        self.layer2 = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu')

    def call(self, inputs, concat_layer):
        _out = self.deconv_layer(inputs)
        _out = tf.keras.layers.concatenate([concat_layer, _out])
        _out = self.dropout(_out)
        _out = self.layer1(_out)
        return self.layer2(_out)


class UnetMiddleLayer(tf.keras.layers.Layer):
    def __init__(self, kernel_size, filters):
        super(UnetMiddleLayer, self).__init__()
        self.layer1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', activation='relu')
        self.layer2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', activation='relu')

    def call(self, inputs):
        _out = self.layer1(inputs)
        return self.layer2(_out)


# Set generators


In [0]:
def get_train_generator(directory, mask_dir, img_dir, read_type=cv2.IMREAD_GRAYSCALE):
    def train_generator():
        _img_dir = join_paths(directory, img_dir)
        _mask_dir = join_paths(directory, mask_dir)

        for imag_file in os.listdir(_img_dir):
            x_img = cv2.imread(join_paths(_img_dir, imag_file), read_type)
            y_img = cv2.imread(join_paths(_mask_dir, imag_file), read_type)
            x_img = np.pad(x_img, ((13, 14), (13, 14)), mode='symmetric').astype(np.float64) / 255.0
            y_img = np.pad(y_img, ((13, 14), (13, 14)), mode='symmetric').astype(np.float64) / 255.0
            img_shape = (x_img.shape[0], x_img.shape[1], 1)
            yield (x_img.reshape(img_shape), y_img.reshape(img_shape))

    return train_generator


def get_test_generator(directory, img_dir, read_type=cv2.IMREAD_GRAYSCALE):
    def test_generator():
        _img_dir = join_paths(directory, img_dir)
        for imag_file in os.listdir(_img_dir):
            x_img = cv2.imread(join_paths(_img_dir, imag_file), read_type)
            yield x_img

    return test_generator


def create_data_set_from_generator(generator, _types, _shapes, buffer_size=100):
    ds = tf.data.Dataset.from_generator(generator, _types, _shapes)
    ds = ds.prefetch(buffer_size)
    return ds

# Model

In [0]:
class UnetModel(tf.keras.Model):
    def __init__(self):
        super(UnetModel, self).__init__()
        kernel_shape = (3, 3)
        filters = 64
        # block1 encoder
        # input_shape = input_shape; ex. (128, 128)
        # output_shape = (input_shape) / 2; ex. (64, 64, filters)
        self.block1_encoder = UnetEncodeLayer(kernel_shape, filters)

        # block2 encoder
        # input_shape = output_shape
        # output_shape = (input_shape) / 2; ex. (32, 32, filters * 2)
        self.block2_encoder = UnetEncodeLayer(kernel_shape, filters * 2)

        # block 3 encoder
        # input_shape = output_shape
        # output_shape = (input_shape) / 2; ex. (16, 16, filters * 4)
        self.block3_encoder = UnetEncodeLayer(kernel_shape, filters * 4)

        # block 4 encoder
        # input_shape = output_shape
        # output_shape = (input_shape) / 2; ex. (8, 8, filters * 8)
        self.block4_encoder = UnetEncodeLayer(kernel_shape, filters * 8)

        # block middle
        # input_shape = output_shape
        # output_shape = (input_shape); ex. (8, 8, filters * 16)
        self.middle_block = UnetMiddleLayer(kernel_shape, filters * 16)

        # block 4 decoder
        # input_shape = output_shape
        # output_shape = (input_shape) * 2; ex. (16, 16, filters * 8)
        self.block4_decoder = UnetDecodeLayer(kernel_shape, filters * 8)

        # block 3 decoder
        # input_shape = output_shape
        # output_shape = (input_shape) * 2; ex. (32, 32, filters * 4)
        self.block3_decoder = UnetDecodeLayer(kernel_shape, filters * 4)

        # block 2 decoder
        # input_shape = output_shape
        # output_shape = (input_shape) * 2; ex. (64, 64, filters * 2)
        self.block2_decoder = UnetDecodeLayer(kernel_shape, filters * 2)

        # block 1 decoder
        # input_shape = output_shape
        # output_shape = (input_shape) * 2; ex. (128, 128, filters)
        self.block1_decoder = UnetDecodeLayer(kernel_shape, filters)

        self.out_layer = tf.keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")

    def call(self, inputs):
        block1_en_out, block1_en_out_pool = self.block1_encoder(inputs)
        block2_en_out, block2_en_out_pool = self.block2_encoder(block1_en_out_pool)
        block3_en_out, block3_en_out_pool = self.block3_encoder(block2_en_out_pool)
        block4_en_out, block4_en_out_pool = self.block4_encoder(block3_en_out_pool)

        middle_out = self.middle_block(block4_en_out_pool)

        block4_de_out = self.block4_decoder(middle_out, block4_en_out)
        block3_de_out = self.block3_decoder(block4_de_out, block3_en_out)
        block2_de_out = self.block2_decoder(block3_de_out, block2_en_out)
        block1_de_out = self.block1_decoder(block2_de_out, block1_en_out)
        return self.out_layer(block1_de_out)


# Main

In [12]:
def build_model(input_layer, start_neurons):
    # 128 -> 64
    conv1 = tf.keras.layers.Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.01))(input_layer)
    conv1 = tf.keras.layers.Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.01))(conv1)
    pool1 = tf.keras.layers.MaxPooling2D((2, 2))(conv1)
    pool1 = tf.keras.layers.Dropout(0.25)(pool1)

    # 64 -> 32
    conv2 = tf.keras.layers.Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.01))(pool1)
    conv2 = tf.keras.layers.Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.01))(conv2)
    pool2 = tf.keras.layers.MaxPooling2D((2, 2))(conv2)
    pool2 = tf.keras.layers.Dropout(0.5)(pool2)

    # 32 -> 16
    conv3 = tf.keras.layers.Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001))(pool2)
    conv3 = tf.keras.layers.Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001))(conv3)
    pool3 = tf.keras.layers.MaxPooling2D((2, 2))(conv3)
    pool3 = tf.keras.layers.Dropout(0.5)(pool3)

    # 16 -> 8
    conv4 = tf.keras.layers.Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.001))(pool3)
    conv4 = tf.keras.layers.Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.001))(conv4)
    pool4 = tf.keras.layers.MaxPooling2D((2, 2))(conv4)
    pool4 = tf.keras.layers.Dropout(0.5)(pool4)

    # Middle
    convm = tf.keras.layers.Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.001))(pool4)
    convm = tf.keras.layers.Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.001))(convm)

    # 8 -> 16
    deconv4 = tf.keras.layers.Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = tf.keras.layers.concatenate([deconv4, conv4])
    uconv4 = tf.keras.layers.Dropout(0.5)(uconv4)
    uconv4 = tf.keras.layers.Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.001))(uconv4)
    uconv4 = tf.keras.layers.Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.001))(uconv4)

    # 16 -> 32
    deconv3 = tf.keras.layers.Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = tf.keras.layers.concatenate([deconv3, conv3])
    uconv3 = tf.keras.layers.Dropout(0.5)(uconv3)
    uconv3 = tf.keras.layers.Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.001))(uconv3)
    uconv3 = tf.keras.layers.Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.001))(uconv3)

    # 32 -> 64
    deconv2 = tf.keras.layers.Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = tf.keras.layers.concatenate([deconv2, conv2])
    uconv2 = tf.keras.layers.Dropout(0.5)(uconv2)
    uconv2 = tf.keras.layers.Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.001))(uconv2)
    uconv2 = tf.keras.layers.Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.001))(uconv2)

    # 64 -> 128
    deconv1 = tf.keras.layers.Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = tf.keras.layers.concatenate([deconv1, conv1])
    uconv1 = tf.keras.layers.Dropout(0.5)(uconv1)
    uconv1 = tf.keras.layers.Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.001))(uconv1)
    uconv1 = tf.keras.layers.Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same",  kernel_regularizer=tf.keras.regularizers.l2(0.001))(uconv1)

    #uconv1 = Dropout(0.5)(uconv1)
    output_layer = tf.keras.layers.Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
    return output_layer

input_layer = tf.keras.layers.Input((128, 128, 1))
output_layer = build_model(input_layer, 16)
model = tf.keras.models.Model(input_layer, output_layer)

model.compile(loss=dice_loss, optimizer="adam", metrics=["accuracy", dice_coef])

ds_train = create_data_set_from_generator(get_train_generator('drive/My Drive/Colab Notebooks/train', mask_dir='masks', img_dir='images'),
                                          _types=(tf.float64, tf.float64),
                                          _shapes=(tf.TensorShape([128, 128, 1]), tf.TensorShape([128, 128, 1])))

early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("./keras.model", save_best_only=True, verbose=1)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5, min_lr=0.00001, verbose=1)
train_ds = train_ds.batch(32)
model.fit(train_ds, epochs = 30, validation_data=valid_ds,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr])

W0808 14:23:49.954592 140305021880192 training_utils.py:1300] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/30


InvalidArgumentError: ignored